In [1]:
# Install required package for running Flask on Google Colab
!pip install flask-ngrok
!pip install pyngrok
!pip install FastAPI
!pip install flask[async]

ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/681.2 kB ? eta -:--:--
     -                                     30.7/681.2 kB 640.0 kB/s eta 0:00:02
     -                                     30.7/681.2 kB 640.0 kB/s eta 0:00:02
     ----                                  81.9/681.2 kB 573.4 kB/s eta 0:00:02
     -----                                112.6/681.2 kB 652.2 kB/s eta 0:00:01
     ----------                           194.6/681.2 kB 841.6 kB/s eta 0:00:01
     ------------                         235.5/681.2 kB 846.9 kB/s eta 0:00:01
     ------------------------               440.3/681.2 kB 1.4 MB/s eta 0:00:01
     ---------------------------            501.8/681.2 kB 1.4 MB/s eta 0:00:01
     -----------------------------------    634.9/681.2 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 681.2/681.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyngr


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/96.1 kB ? eta -:--:--
     --------------------------------------   92.2/96.1 kB 1.7 MB/s eta 0:00:01
     ---------------------------------------- 96.1/96.1 kB 1.8 MB/s eta 0:00:00
                                              0.0/242.2 kB ? eta -:--:--
     ---------------------------            174.1/242.2 kB 5.3 MB/s eta 0:00:01
     -------------------------------------- 242.2/242.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.22 MiB | 34.24 MiB/s, done.
Resolving deltas: 100% (513/513), done.


In [ ]:
!pip install -r /content/yolov7/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.7 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import Response
import os
import shutil
from subprocess import Popen
# from arg_parsers import parse_args
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

import sys
sys.path.insert(0, './yolov7')

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

In [ ]:
"""
Setup and cleanup code to get the data zip files from Google Drive,
and extract them for use from this Google Colab Notebook.
"""
def extractzip(zip_path, extract_path):
  import os
  import zipfile

  # print('zipfile: ', zip_path)
  # print('extract path: ', extract_path)

  os.makedirs(extract_path, exist_ok = True)

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

def setup_data():
  # MOUNT_PATH's value is where the drive gets mounted by convention.
  # From there onwards, the file paths point to where the zip files are on
  # my Google Drive.
  # The zip file will be extracted to EXTRACT_PATH. We can then see in
  # colab using the Files view on the left.
  MOUNT_PATH       = "/content/drive"
  BDD_ZIPFILE_PATH = MOUNT_PATH + "/MyDrive/AIML_HYD_20/Capstone_Project/FinalWeights/best.pt"
  EXTRACT_PATH     = "/content/yolov7"

  from google.colab import drive
  drive.mount(MOUNT_PATH)
  shutil.copy(BDD_ZIPFILE_PATH, EXTRACT_PATH)
  drive.flush_and_unmount()

In [ ]:
setup_data()

Mounted at /content/drive


In [ ]:
def read(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines[0]

def parse_args():
    opt = {
        'weights'     : '/content/yolov7/best.pt',
        'source'      : "",  # file/folder, 0 for webcam
        'img_size'    : 640,
        'conf_thres'  : 0.25,
        'iou_thres'   : 0.45,
        'device'      : '',
        'view_img'    : False,
        'save_txt'    : False,
        'save_conf'   : False,
        'nosave'      : False,
        'classes'     : None,
        'agnostic_nms': False,
        'augment'     : False,
        'update'      : False,
        'project'     : 'runs/detect',
        'name'        : 'exp',
        'exist_ok'    : False,
        'no_trace'    : False,
    }

    return opt

In [ ]:
opt = None
source, model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify = None, None, None, None, None, None, None, None, None, None, None
opt = parse_args()

def yolo_load():
    global model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify
    weights, view_img, save_txt, imgsz, trace = opt['weights'], opt['view_img'], opt['save_txt'], opt['img_size'], not opt['no_trace']

    # Directories
    save_dir = Path(increment_path(Path(opt['project']) / opt['name'], exist_ok=opt['exist_ok']))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(opt['device'])
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size

    if trace:
        model = TracedModel(model, device, opt['img_size'])

    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

    # return model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify


def yolo_detect(input_image_path = ""):
    global model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify

    # Directories
    save_dir = Path(increment_path(Path(opt['project']) / opt['name'], exist_ok=opt['exist_ok']))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    source = input_image_path
    # source = source

    save_img = not opt['nosave'] and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment=opt['augment'])[0]

        # Inference
        t1 = time_synchronized()
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = model(img, augment=opt['augment'])[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, opt['conf_thres'], opt['iou_thres'], classes=opt['classes'], agnostic=opt['agnostic_nms'])
        t3 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if opt['save_conf'] else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)

            # Print time (inference + NMS)
            print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS', flush=True)

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                    print(f" The image with the result is saved in: {save_path}")
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        #print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')


yolo_load()

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



In [ ]:
import os
from flask import Flask, request, render_template, Response
from werkzeug.utils import secure_filename
from pyngrok import ngrok  # Import ngrok package

# Set up Flask app
app = Flask(__name__, static_folder='/content')

# UPLOAD_FOLDER = '/content/uploaded_images'
# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
# app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg', 'gif', 'mp4'}

# Define allowed file extensions
# def allowed_file(filename):
#     return '.' in filename and \
#            filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

# this is our start page
@app.route("/")
async def homepage():
    html_content = f"""
        <html>
            <body style="text-align:center">
                <h1>Road Object Detection</h1>
                <p>This page serves a "custom" <a href="https://arxiv.org/abs/2207.02696">Yolov7 model</a> for Road Object Detection.</p>
                <p>It has been trained on the <a href="https://bdd-data.berkeley.edu/">BDD100K dataset</a>.</p>
                <p>Choose an image or video, and click Detect.</p>
                <form action="/detect" method="post" enctype="multipart/form-data">
                    <input type="file" name="inputfile">
                    <input type="submit" value="Detect">
                </form>
            </body>
        </html>
        """

    print(opt)
    global model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify
    # model, stride, imgsz, device, half, view_img, save_txt, save_dir, modelc, classify =
    return html_content

@app.route("/detect", methods=['POST'])
async def detect():
    # ------------------------------------------------------
    # 1. proceed only if the file has an accepted extension
    video     = False
    inputfile = request.files['inputfile']
    fname     = secure_filename(inputfile.filename)

    if fname.endswith(('.jpg', '.jpeg', '.png')):
        video = False
    elif fname.endswith(('mp4')):
        video = True
    else:
        html_content = f"""
        <html>
            <body>
                <p><b>error: unsupported format.</b></p>
                <p>Supported formats are .jpg, .jpeg, .png, .mp4</p>
            </body>
        </html>
        """
        return html_content

    # ------------------------------------------------------
    # 2. upload input file
    upload_folder = "/content/uploaded_images"
    os.makedirs(upload_folder, exist_ok=True)
    input_file_path = os.path.join(upload_folder, fname)
    inputfile.save(input_file_path)
    # ------------------------------------------------------

    # ------------------------------------------------------
    # 3. run detection on the input file.
    #
    # To make it easy to report results we will clean up the results
    # folder if it exits.
    # Note that we should have already cloned the yolov7 repo at the
    # current folder. Therefore the results will be emitted to:
    # "yolov7/runs"
    runs_folder = "/content/runs"
    if os.path.exists(runs_folder):
        process = Popen(["rm", "-r", runs_folder])
        process.wait()  # Wait for the process to complete

    # we will create a log text file and redirect stdout there.
    # The detections will be reported in this text filem
    log_folder    = "logs"
    log_filename  = "log.txt"
    os.makedirs(log_folder, exist_ok = True)
    log_file_path = os.path.join(log_folder, log_filename)
    logfile       = open(log_file_path, "w")

    import sys
    sys.stdout = logfile

    # wt_file = "yolov7/best.pt"
    # process = Popen(["python", "yolov7/detect.py", "--weights", wt_file,
    #                   "--source", input_file_path])
    # process.wait()

    with torch.no_grad():
        yolo_detect(input_file_path)

    # ------------------------------------------------------

    # ------------------------------------------------------
    # 4. get path to the predicted file, and to the results.
    output_folder        = runs_folder + "/detect/exp"
    prediction_filename  = os.listdir(output_folder)[0]
    prediction_file_path = os.path.join(output_folder, prediction_filename)

    results = read(log_file_path)
    # ------------------------------------------------------

    # from fastapi.staticfiles import StaticFiles
    # app.mount("uploaded_images", StaticFiles(directory="uploaded_images"), name="uploaded_images")
    # app.mount("runs", StaticFiles(directory="runs"), name="runs")

    # ------------------------------------------------------
    # 5. return the predicted file.
    if video:
        html_content = f"""
        <html>
            <body>
                <p><b>Chosen video: </b>{fname}</p>
                <video controls>
                    <source src="{input_file_path}">
                </video>
                <br>
                <br>
                <p><b>Please download video to view detections: </b>
                    <a href="{prediction_file_path}" download>Download Video</a>
                </p>
            </body>
        </html>
        """
        return html_content

    html_content = f"""
    <html>
        <body>
            <p><b>Chosen image: </b>{fname}</p>
            <img id="image1" src="{input_file_path}" alt="Image will display here.">
            <br>
            <br>
            <p><b>Image with detections:</b> {results}</p>
            <img id="image2" src="{prediction_file_path}" alt="Image will display here.">
        </body>
    </html>
    """
    return html_content
    # ------------------------------------------------------
# ----------------------------------------------------------


# Start ngrok tunnel with your authtoken
authtoken = "2SjjXeH0EcSRIVmHFOkD0CINQWs_7hUDVDn8tLhCZBKWonUR"  # Replace with your ngrok authtoken
ngrok.set_auth_token(authtoken)
tunnel = ngrok.connect(5000)
print(f"Public URL: {tunnel.public_url}")

# Run the Flask app
app.run()


Public URL: https://3adb-34-32-198-179.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:24:37] "GET / HTTP/1.1" 200 -


{'weights': '/content/yolov7/best.pt', 'source': '', 'img_size': 640, 'conf_thres': 0.25, 'iou_thres': 0.45, 'device': '', 'view_img': False, 'save_txt': False, 'save_conf': False, 'nosave': False, 'classes': None, 'agnostic_nms': False, 'augment': False, 'update': False, 'project': 'runs/detect', 'name': 'exp', 'exist_ok': False, 'no_trace': False}


INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:24:38] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:24:50] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:24:50] "GET /content/uploaded_images/ac9be3fe-790d1f8e.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:24:50] "GET /content/runs/detect/exp/ac9be3fe-790d1f8e.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:25:06] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:25:29] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:25:30] "GET /content/uploaded_images/traffic.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:25:34] "GET /content/runs/detect/exp/traffic.mp4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Aug/2023 19:26:17] "GET / HTTP/1.1" 200 -
